In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [12]:
# Đọc file CSV chứa link phim
input_file = "C:/thaiduong/SocialNetwork/rottentomatoes_movie_links.csv"  # Đường dẫn đến file chứa cột "Movie Links"
df_links = pd.read_csv(input_file)
movie_links = df_links["Movie Links"].tolist()

# Danh sách để lưu dữ liệu phim
movie_data = []

# Duyệt qua từng link phim
for link in movie_links:
    print(f"Đang lấy dữ liệu từ: {link}")
    try:
        # Gửi request
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(link, headers=headers)
        if response.status_code != 200:
            print(f"Không thể kết nối tới {link}")
            continue

        # Phân tích HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # Thu thập dữ liệu
        movie_info = {}

        # 0. Tên phim
        try:
            movie_title = soup.select_one('rt-text[slot="title"]').text.strip()
            movie_info["Movie Title"] = movie_title
        except AttributeError:
            movie_info["Movie Title"] = None

        # 1. Điểm Tomatometer
        try:
            tomatometer = soup.select_one('rt-text[slot="criticsScore"]').text.strip()
            movie_info["Tomatometer"] = tomatometer
        except AttributeError:
            movie_info["Tomatometer"] = None

        # 2. Điểm Popcornmeter
        try:
            popcornmeter = soup.select_one('rt-text[slot="audienceScore"]').text.strip()
            movie_info["Popcornmeter"] = popcornmeter
        except AttributeError:
            movie_info["Popcornmeter"] = None

        # 3. Movie Info (category-wrap)
        for category in soup.select("div.category-wrap"):
            try:
                # Lấy key (tên cột)
                key_tag = category.select_one('rt-text[class="key"]')
                if key_tag:
                    key = key_tag.text.strip()
                else:
                    key = (
                        category.select_one("rt-text.key").text.strip()
                        if category.select_one("rt-text.key")
                        else "Unknown"
                    )

                # Lấy tất cả các value từ rt-text[data-qa="item-value"]
                values = []

                # Trường hợp có nhiều value trong rt-text[data-qa="item-value"]
                value_tags = category.select('rt-text[data-qa="item-value"]')
                if value_tags:
                    values.extend(
                        [
                            value.text.strip()
                            for value in value_tags
                            if value.text.strip()
                        ]
                    )

                # Nếu không tìm thấy trong rt-text[data-qa="item-value"], kiểm tra rt-link
                if not values:
                    value_link_tags = category.select("rt-link")
                    if value_link_tags:
                        values.extend(
                            [
                                value.text.strip()
                                for value in value_link_tags
                                if value.text.strip()
                            ]
                        )

                # Lưu key và value vào dictionary
                # Gộp các giá trị thành chuỗi nếu có nhiều giá trị
                movie_info[key] = ", ".join(values) if values else None

            except Exception as e:
                print(f"Lỗi khi xử lý category: {e}")
                continue

        # 4. Cast (diễn viên)
        cast_list = []
        for cast in soup.select('p.name[data-qa="person-name"]'):
            cast_name = cast.text.strip()
            cast_list.append(cast_name)
        movie_info["Cast"] = ", ".join(cast_list)  # Lưu diễn viên thành chuỗi

        # Lưu dữ liệu phim
        movie_data.append(movie_info)

        # Tạm dừng để tránh bị chặn
        time.sleep(1)

    except Exception as e:
        print(f"Lỗi khi xử lý link {link}: {e}")

# Chuyển dữ liệu thành DataFrame và lưu vào file CSV
output_df = pd.DataFrame(movie_data)


Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/john_wick
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/red_army_2015
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/roman_holiday
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/the_merry_gentlemen
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/laroy_texas
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/ghostbusters_frozen_empire
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/princess_mononoke
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/1018688-shadow_of_a_doubt
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/no_no_a_dockumentary
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/fear_of_fear
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/caddo_lake
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/star_wars_the_rise_of_skywalker
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/american_fiction
Đang lấy dữ liệu từ: https://www.rottentomatoes.com/m/cabrini
Đ

In [14]:
# Tạo DataFrame từ dữ liệu phim


# Lưu vào file CSV
output_file = "movie_dataset.csv"
output_df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"Dữ liệu đã được lưu vào {output_file}")

Dữ liệu đã được lưu vào movie_dataset.csv
